In [61]:
import os
from langchain_ollama import ChatOllama

# Initialize an Ollama Client for our generative Llm model

text_model = "llama3.2"

def llm_client_loader():
    """This function serves an Ollama-hosted text generator model, to be used by our graphs."""
    try:
        llm = ChatOllama(
            model=text_model,
            temperature=0.2
        )
        return llm
    except Exception as e:
        print(f"Error {e} instantiating the Ollama client, is the Ollama server running?.")

llm = llm_client_loader()

In [156]:
stock_symbol = "AAPL" #relevant later

In [159]:
from langchain_community.tools import DuckDuckGoSearchRun
# first an orchestrator generates a research plan
import os
from langgraph.constants import START
from langgraph.graph import StateGraph
from langgraph.graph import END
from pydantic import Field, BaseModel
from langgraph.graph import MessagesState
from typing import Annotated, Literal
import operator
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.types import Send
from typing import List, TypedDict

# Schemas for structured output
class ReportSection(BaseModel):
    research_objective: str = Field(description="The descriptive title and a few keywords for this section.")

# messages + structured output
class FinancialReport(BaseModel):
    sections: List[ReportSection] = Field(description="Sections of the financial report.")

# Internal states (dynamic)
# Augment the LLM with schema for structured output
planner = llm.with_structured_output(FinancialReport)

# Internal State definition

class State(MessagesState):
    stock_symbol: str  # the stock to be analyzed
    report_sections: list[ReportSection]  # list of Report sections to be filled out after research
    completed_analyses: Annotated[
        list, operator.add
    ]  # Shared key for the analysts to write to
    relevant_or_not: Annotated[list, operator.add] #Literal["Relevant", "Not relevant"]
    filtered_analyses: Annotated[
        list, operator.add
    ]
    macro_financial_report: str

class WorkerState(TypedDict):
    report_section: ReportSection
    completed_analyses: Annotated[list, operator.add]  # keys must match with other State!
    relevant_or_not: Annotated[list, operator.add] #Literal["Relevant", "Not relevant"]
    filtered_report: str
    filtered_analyses: Annotated[list, operator.add]
    final_filtered_analysis: list

# Nodes / Tools
def orchestrator(state: State):
    try:
        """Orchestrator that instantiates a research plan in specific sections in order to obtain a comprehensive overview of the selected stock."""

        # Generate queries...
        report_sections = planner.invoke(
            [
                SystemMessage(
                    content="You are tasked with generating a comprehensive, deep research initiative plan that synthesizes expertise from multiple domain experts to develop a robust understanding of the qualitative and quantitative behavior of the given stock. Provide a descriptive topic to research as well as a few keywords for each corresponding section. The output should be formatted as if it was ready to be ingested by a search engine. Example: Apple new CEO company turmoil."),
                HumanMessage(content=f"{state["stock_symbol"]}."),
            ]
        )

        return {"report_sections": report_sections.sections}

    except Exception as e:
        print(f"Error {e} during the orchestrator process.")

def llm_call(state: WorkerState):
    """Worker performs research on the given research objective. If it must use a tool (it waits until it is redirected to use the tool
    """

    try:
        print(f"Worker instantiated: {state['report_section'].research_objective}.")
        research_section_result = DuckDuckGoSearchRun()
        response = research_section_result.invoke(state['report_section'].research_objective)

        # Write the search result.
        return {"completed_analyses": [response]}

    except Exception as e:
        print(f"Error {e} during the llm-call process.")

# evaluator, after the Llm call

# Schema for structured output to use in evaluation
class Evaluation(BaseModel):
    grade: Literal["Relevant", "Not relevant"] = Field(
        description=f"Decide whether the content in the section is somehow connected to the given Stock {stock_symbol}, the overall financial market, or is not connected/relevant.",
    )

# Augment the LLM with schema for structured output
evaluator = llm.with_structured_output(Evaluation)

def llm_call_evaluator(state: WorkerState):
    """LLM evaluates whether the information is connected to the given stock or financial market, or is not connected/relevant."""

    grade = evaluator.invoke(f"Grade the information with relevant if it mentions the given stock {stock_symbol} or financial market, and not relevant if it doesn't: {state['filtered_report']}")

    print(f"LLM evaluated prompt:{state['filtered_report']}")
    print(f"LLM evaluation result: {grade}")

    if grade.grade == "Relevant":
        return {"filtered_analyses": [state['filtered_report']]}
    else:
        return None

def synthesizer(state: State):
        """Synthesize a summary report from the collection of news articles. Don't forget to include the
        macroeconomic label for each news article."""
        try:
            # List of completed sections
            separate_analyses = state["filtered_analyses"]

            final_response = llm.invoke(f"The following is set of summaries: {separate_analyses} "
                                                    f"Take these and distill them into a final executive summary of the "
                                                    f"main themes, capturing the key ideas without missing critical "
                                                    f"points. Ensure the summary touches upon all of the main themes "
                                                    f"found, and be sure to include important details.")

            #print(f"Synthesizing summary report: {completed_report}")
            # Format completed section to str to use as context for final sections
            #serialized_completed_report = "\n\n---\n\n".join(completed_report)

            return {"macro_financial_report": final_response}
        except Exception as e:
            print(f"Error {e} during the synthesizer process.")

# a spawner generates llm_call functions
# Conditional edge function to create llm_call workers that each write a section of the report
def assign_workers(state: State):
    try:
        """Assign a worker to each report section."""
        # Kick off section writing in parallel via Send() API
        return [Send("llm_call", {"report_section": s}) for s in state["report_sections"]]
    except Exception as e:
        print(f"Error {e} during the worker assignment process 1.")

# a spawner generates llm_evaluator functions
# Conditional edge function to create llm_call workers that each evaluate a section of the report
def assign_workers_filter(state: State):
    try:
        """Assign a worker to each report section."""
        # Kick off section writing in parallel via Send() API
        return [Send("llm_call_evaluator", {"filtered_report": s}) for s in state["completed_analyses"]]
    except Exception as e:
        print(f"Error {e} during the worker assignment process 2.")

In [ ]:
class ReportSectionW(BaseModel):
    title: str = Field(description="The title that best fits the information.")
    summary: str = Field(description="A bullet point summary of the research.")

summarizer = llm.with_structured_output(ReportSectionW) # yfinance

In [160]:
# Build workflow
orchestrator_worker_builder = StateGraph(State)

# Add the nodes
orchestrator_worker_builder.add_node("orchestrator", orchestrator)
orchestrator_worker_builder.add_node("llm_call", llm_call)
orchestrator_worker_builder.add_node("llm_call_evaluator", llm_call_evaluator)
orchestrator_worker_builder.add_node("synthesizer", synthesizer)

# Add edges to connect nodes
orchestrator_worker_builder.add_edge(START, "orchestrator")
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator", assign_workers, ["llm_call"]
)
# spawn? or one-to-one is enough?
orchestrator_worker_builder.add_conditional_edges(
    "llm_call", assign_workers_filter, ["llm_call_evaluator"]
)
orchestrator_worker_builder.add_edge("llm_call_evaluator", "synthesizer")
orchestrator_worker_builder.add_edge("synthesizer", END)

# Compile the workflow
graph = orchestrator_worker_builder.compile()

missing running the function below to get the graph visualization

In [ ]:
def _repr_mimebundle_(self, **kwargs):
    """Mime bundle used by jupyter to display the graph"""
    output = {
        "text/plain": repr(self),
        "image/png": self.get_graph().draw_mermaid_png()
        }
    return output

graph._repr_mimebundle_ = _repr_mimebundle_.__get__(graph)
print(graph)
display(graph)

In [161]:
state = graph.invoke({"stock_symbol": stock_symbol})

Worker instantiated: Analyzing the Role of Artificial Intelligence in Apple's (AAPL) Product Development and Innovation Strategy.Worker instantiated: Understanding the Impact of Apple's (AAPL) Recent Earnings Report on Investor Sentiment and Stock Price Volatility.

Worker instantiated: Investigating the Relationship Between Apple's (AAPL) Supply Chain Management and Environmental Sustainability.
LLM evaluated prompt:Sep 11, 2025 · Using a VPN, or virtual private network, is one of the best ways to protect your online privacy. We review dozens every year, and these are the best VPNs we've tested. Proton VPN rises above the competition with an excellent collection of features, a high-performance server network, and a nearly peerless free subscription option, making it the top … Fast, secure, and risk-free VPN for online privacy. Encrypt your traffic, change your IP, and browse freely without restrictions or intrusive ads. Protect your privacy, stream worldwide, and enjoy fast speeds wit

In [167]:
print(state["macro_financial_report"].content)

Here is a distilled executive summary of the main themes:

**Apple Stock: Volatility Ahead**

* Apple's shares have declined by 18% from peak valuations, indicating potential investor skepticism.
* Technical indicators are converging, suggesting possible volatility akin to a tightly wound spring poised for release.

**Recent Earnings Report: Mixed Signals**

* Analysts were cautious heading into Apple's latest earnings report but were relieved to see the company post its biggest quarterly revenue growth since [date].
* The "Magnificent Seven" titan's strong brand and innovations in custom silicon are being weighed against geopolitical and operational hurdles that present a bearish case for AAPL stock.

**Key Factors to Watch**

* The staggered launch of Apple Intelligence is expected to result in a pickup in demand later this year, according to Bank of America analyst Wamsi Mohan.
* Geopolitical and operational challenges are affecting the company's performance.

**Industry Context: St